<a href="https://colab.research.google.com/github/146790g/VGG19-BreakHis/blob/master/BreakHis_VGG19_prediction_%E8%A3%9C%E8%B6%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 深層学習 VGG19モデルを用いた乳がん染色画像（BreakHis）の解析のための準備
#（その2 : Visual Predictive Check 補足編）
##  Deep Learning for Magnification Independent Breast Cancer Histopathology
## Image Classification using VGG19 Model - Visual Predictive Check



<h2>対象データは、こちらをクリック</>
<a href='https://www.kaggle.com/ambarish/breakhis'>Kaggle BreakHis Data</a>

４つのMagnification levelのデータをすべて合算して、解析する

<br>

## 4 Magnification Levels: ×40, ×100, ×200, ×400


During analysis and annotation, pathologists starts by identifying ROIs in
the lowest magnification level slide (×40), then dives deeper in the latter using higher magnification levels (×100, ×200) until having a profound insight
(×400). 

<br>

## データ数 の概要

・Training Data: 5026 images <br>
  -- Benign Data: 1472 images <br>
  -- Mallignant Data: 3554 images 

・Validation Data: 2904 images <br>
  -- Benign Data: 1008 images <br>
  -- Mallignant Data:1896 images <br>

## Data Set Details by Magnification levels.

 ×40 : <br>
×100 : <br>
×200 : <br>
×400 : <br>

In [1]:
%tensorflow_version 2.x

In [9]:
%pwd
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
!ls -lh '/content/gdrive/My Drive/Colab Notebooks/BreakHis/data'

total 8.0K
drwx------ 2 root root 4.0K Jul 21 03:42 train
drwx------ 2 root root 4.0K Jul 27 01:49 validation


In [11]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator,load_img
import numpy as np
import os
#import Pillow
#from PIL import Image
#import opencv
from PIL import Image
from keras.preprocessing.image import load_img, save_img, img_to_array, array_to_img
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, initializers

from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import numpy as np


train_data_dir = '/content/gdrive/My Drive/Colab Notebooks/BreakHis/data/train' # Training Data のフォルダー
validation_data_dir = '/content/gdrive/My Drive/Colab Notebooks/BreakHis/data/validation' # Validation Data のフォルダー
result_dir = '/content/gdrive/My Drive/Colab Notebooks/BreakHis/results'        # VGG19の出力結果を保存するフォルダー
test_dir = '/content/gdrive/My Drive/Colab Notebooks/BreakHis/test'
    


In [15]:
   
normal_files=os.listdir(os.path.join(test_dir,'benign'))

filename=os.path.join(test_dir,'benign',normal_files[0])

print(filename)
  
img_h, img_w =400, 400
    
channels = 3

img = image.load_img(
    filename,
    target_size=(img_h, img_w))

x = image.img_to_array(img)

x = np.expand_dims(x, axis=0)

x = x / 255.0

print(x)
print(type(x))
print(x.shape)

    

/content/gdrive/My Drive/Colab Notebooks/BreakHis/test/benign/SOB_B_A-14-22549G-200-029.png
[[[[0.827451   0.8235294  0.8039216 ]
   [0.84313726 0.84313726 0.8117647 ]
   [0.84313726 0.8235294  0.8117647 ]
   ...
   [0.7764706  0.67058825 0.8       ]
   [0.7490196  0.6509804  0.7647059 ]
   [0.6666667  0.60784316 0.69803923]]

  [[0.83137256 0.827451   0.8117647 ]
   [0.84313726 0.8392157  0.81960785]
   [0.85490197 0.8352941  0.8235294 ]
   ...
   [0.7529412  0.64705884 0.7764706 ]
   [0.7411765  0.6431373  0.75686276]
   [0.67058825 0.6        0.69411767]]

  [[0.8352941  0.83137256 0.8156863 ]
   [0.85490197 0.8509804  0.83137256]
   [0.8509804  0.83137256 0.81960785]
   ...
   [0.7294118  0.62352943 0.74509805]
   [0.7254902  0.6313726  0.73333335]
   [0.65882355 0.5882353  0.6901961 ]]

  ...

  [[0.84313726 0.87058824 0.8392157 ]
   [0.8509804  0.88235295 0.83137256]
   [0.85882354 0.88235295 0.84313726]
   ...
   [0.85882354 0.8235294  0.8352941 ]
   [0.8117647  0.78431374 0.815

##第１章　Visual Predictive Check


In [5]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing import image
import numpy as np
from keras.models import load_model

model=load_model(os.path.join(result_dir,'breakhis_vgg19model_parameter.h5'))



model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 12, 12, 512)       20024384  
_________________________________________________________________
flatten (Flatten)            (None, 73728)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               18874624  
_________________________________________________________________
sigmoid (Dense)              (None, 1)                 257       
Total params: 38,899,265
Trainable params: 28,314,113
Non-trainable params: 10,585,152
_________________________________________________________________


## モデルの読み込み 

In [18]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.applications.vgg19 import VGG19
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing import image
import numpy as np
from keras.models import load_model


pred = model.predict(x)

print(pred)


[[0.9990202]]
